In [1]:
import requests
import numpy as np
import pandas as pd

from scipy.sparse import hstack

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import Ridge
from sklearn.feature_extraction import DictVectorizer

In [2]:
# Выбираем профессии, которые будем добавлять в датафрейм
jobs = ['Data Scientist', 'Data Analyst']
vacancies = pd.DataFrame(columns=['employment', 'description', 'salary'])
ids = []

# Для каждой профессии отбираем только те, где указана зарплата: 'only_with_salary' : True
for job in jobs:
    url = 'https://api.hh.ru/vacancies'
    params = {'text' : job,
              'only_with_salary' : True,
              'page' : 0,
              'per_page' : 100}

    st = requests.get(url, params).json()
    # И по каждому id добавляем в датафрейм тип занятости, описание и зарплату
    for i in range(len(st['items'])):
        id = st['items'][i]['id']
        if id in ids:
            continue
        ids.append(id)
        vacancy = requests.get("https://api.hh.ru/vacancies/" + id + "?host=hh.ru").json()
        # Нас интересуют только те, где зарплата указана в рублях. Остальные пропускаем.
        # Иначе цены будут в разных измерениях.
        # При желании, можно приделать конвертацию по текущему курсу, но сейчас и так сойдёт)
        if vacancy['salary']['currency'] != 'RUR':
            continue
        # Цены объявлены либо в конкретном диапазоне, либо "от", либо "до"
        # Если диапазон не указан, то запишем крайнее значение
        # Иначе, просто подсчитаем среднюю
        if vacancy['salary']['from'] is None:
            salary = int(vacancy['salary']['to'])
        elif vacancy['salary']['to'] is None:
            salary = int(vacancy['salary']['from'])
        else:
            salary = (int(vacancy['salary']['from']) + int(vacancy['salary']['to'])) // 2
        vac_data = pd.DataFrame([[vacancy['employment']['id'],
                                  vacancy['description'],
                                  salary]],
                                columns=['employment', 'description', 'salary'])
        vacancies = vacancies.append(vac_data, ignore_index=True)

In [3]:
# Теперь создаём датафрейм для интересующих нс вакансий без указания зарплаты
# Я взял с указанной для того, чтобы можно было проверить точность предскзаний
# Просто не буду эту ЗП записывать
target_vacancies = pd.DataFrame(columns=['employment', 'description', 'salary'])
target_ids = ['38696758', '37080920']

for target_id in target_ids:
    vacancy = requests.get("https://api.hh.ru/vacancies/" + target_id + "?host=hh.ru").json()
    vac_data = pd.DataFrame([[vacancy['employment']['id'],
                              vacancy['description'],
                              'NaN']],
                            columns=['employment', 'description', 'salary'])
    target_vacancies = target_vacancies.append(vac_data, ignore_index=True)

In [4]:
# Нас интересуют только слова, поэтому убираем все лишние символы при помощи регулярных выражений,
# а так же приводим все описания к нижнему регистру

vacancies.description = vacancies['description'].apply(lambda x: x.lower())
vacancies.description = vacancies['description'].replace('<[/\w]+>', ' ', regex = True)
vacancies.description = vacancies['description'].replace('[^ЁёА-яa-zA-Z0-9]', ' ', regex = True)

target_vacancies.description = target_vacancies['description'].apply(lambda x: x.lower())
target_vacancies.description = target_vacancies['description'].replace('<[/\w]+>', ' ', regex = True)
target_vacancies.description = target_vacancies['description'].replace('[^ЁёА-яa-zA-Z0-9]', ' ', regex = True)

In [5]:
# Получаем чистое описание
target_vacancies['description'][0]

' мы занимается разработкой и внедрением программных продуктов с использованием компьютерного зрения и машинного обучения    наш стек  python  c   opencv  pytorch  pandas  tensorflow  openvino  docker    задачи       разработать систему сбора данных и обучения на их основе пайплайна распознавания изображений    сейчас это работает для россии  и нужно будет это масштабировать на весь мир        требования       понимание как хранить данные  и как их обрабатывать    как лучше размечать данные яндекс толока  supervise ly  cvat         как минимум поверхностное понимание нейронных сетей    опыт коммерческой разработки связанной с глубоким обучением    документирование процессов    желательно шарить в nosql базах      условия       работа по тк рф  100  белая зп    гибкий график работы    дружный коллектив единомышленников    возможна удаленка   '

In [6]:
# Применяем TF-IDF для преобрзования описаний в векторы признаков
# И при помощи метода min_df оставляем только те, слова, которые встречаются хотя бы в 5-ти документах
vectorizer = TfidfVectorizer(min_df=5)
X_train_vec = vectorizer.fit_transform(vacancies['description'])
X_test_vec = vectorizer.transform(target_vacancies['description'])

In [7]:
# Применяем one-hot-encoding для типа занятости
enc = DictVectorizer()
X_train_categ = enc.fit_transform(vacancies[['employment']].to_dict('records'))
X_test_categ = enc.transform(target_vacancies[['employment']].to_dict('records'))

In [8]:
# Объединим всё получившееся в одну матрицу 'объекты-признаки'
X_for_train = hstack([X_train_vec, X_train_categ])
X_for_test = hstack([X_test_vec, X_test_categ])

In [9]:
# Применим гребневую регрессию для обучения
ridge = Ridge(alpha=1, random_state=42)

In [10]:
# Выделим таргеты в отдельный массив
y_for_train = vacancies['salary'].values

In [11]:
# Фитим регрессию нашими признаками: X_train и ответами(таргетами): y_train
ridge.fit(X_for_train, y_for_train)

Ridge(alpha=1, copy_X=True, fit_intercept=True, max_iter=None, normalize=False,
      random_state=42, solver='auto', tol=0.001)

In [12]:
# Теперь можно делать предсказание по интересующим нас описаниям
ridge.predict(X_for_test)

array([172260.65618197, 152617.81816263])